In [1]:
#all imports here
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/train-images-idx3-ubyte


In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
])


In [4]:
# Load FashionMNIST Dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

100%|██████████| 26.4M/26.4M [00:01<00:00, 14.1MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.89MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.1MB/s]


In [5]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# Class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [7]:
# CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 5 * 5, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

model = CNNModel().to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 1000 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Validation loop
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Save model
    torch.save(model.state_dict(), "/kaggle/working/fashion_mnist_model.pth")


Epoch [1/20], Step [0/1875], Loss: 2.2995
Epoch [1/20], Step [1000/1875], Loss: 0.3610
Validation Loss: 0.3657, Validation Accuracy: 0.8720
Epoch [2/20], Step [0/1875], Loss: 0.4658
Epoch [2/20], Step [1000/1875], Loss: 0.2128
Validation Loss: 0.3244, Validation Accuracy: 0.8852
Epoch [3/20], Step [0/1875], Loss: 0.3292
Epoch [3/20], Step [1000/1875], Loss: 0.1013
Validation Loss: 0.2853, Validation Accuracy: 0.8982
Epoch [4/20], Step [0/1875], Loss: 0.1630
Epoch [4/20], Step [1000/1875], Loss: 0.1324
Validation Loss: 0.2765, Validation Accuracy: 0.9002
Epoch [5/20], Step [0/1875], Loss: 0.1474
Epoch [5/20], Step [1000/1875], Loss: 0.0859
Validation Loss: 0.2960, Validation Accuracy: 0.8947
Epoch [6/20], Step [0/1875], Loss: 0.1065
Epoch [6/20], Step [1000/1875], Loss: 0.2159
Validation Loss: 0.2574, Validation Accuracy: 0.9116
Epoch [7/20], Step [0/1875], Loss: 0.0870
Epoch [7/20], Step [1000/1875], Loss: 0.1616
Validation Loss: 0.2520, Validation Accuracy: 0.9120
Epoch [8/20], Step [